Gradient Boosting Algorithm

In [21]:
# Install dependencies
%pip install pandas
%pip install scikit-learn
%pip install matplotlib


[notice] A new release of pip is available: 25.0 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip



Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.0 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.0 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [22]:
# Import dependencies
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler, LabelEncoder

In [23]:
# Import data
data = pd.read_csv("original_data_with_fraud.csv")

In [24]:
# Data Preprocessing

# Drop columns that are not needed for the model
data = data.drop(columns=['TransactionID', 'AccountID', 'DeviceID', 'IP Address', 'MerchantID'])

# Preprocessing: Handle datetime columns
data['TransactionDate'] = pd.to_datetime(data['TransactionDate'])
data['PreviousTransactionDate'] = pd.to_datetime(data['PreviousTransactionDate'])
data['TimeSinceLastTransaction'] = (data['TransactionDate'] - data['PreviousTransactionDate']).dt.total_seconds()

# Identify numeric and categorical columns
numeric_cols = ['TransactionAmount', 'TransactionDuration', 'LoginAttempts', 'AccountBalance', 'CustomerAge', 'TimeSinceLastTransaction']
categorical_cols = ['TransactionType', 'Location', 'Channel', 'CustomerOccupation']

# Normalize numeric columns
scaler = StandardScaler()
data[numeric_cols] = scaler.fit_transform(data[numeric_cols])

label_encoder = LabelEncoder()
for col in categorical_cols:
    data[col] = label_encoder.fit_transform(data[col])

data.head()
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2512 entries, 0 to 2511
Data columns (total 13 columns):
 #   Column                    Non-Null Count  Dtype         
---  ------                    --------------  -----         
 0   TransactionAmount         2512 non-null   float64       
 1   TransactionDate           2512 non-null   datetime64[ns]
 2   TransactionType           2512 non-null   int64         
 3   Location                  2512 non-null   int64         
 4   Channel                   2512 non-null   int64         
 5   CustomerAge               2512 non-null   float64       
 6   CustomerOccupation        2512 non-null   int64         
 7   TransactionDuration       2512 non-null   float64       
 8   LoginAttempts             2512 non-null   float64       
 9   AccountBalance            2512 non-null   float64       
 10  PreviousTransactionDate   2512 non-null   datetime64[ns]
 11  Fraud                     2512 non-null   bool          
 12  TimeSinceLastTransac

In [25]:
# Gradient Boosting Classification
# Import models and utility functions
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix


# Importing the dataset
X = data.drop(columns=['Fraud', 'TransactionDate', 'PreviousTransactionDate'])
y = data['Fraud']

# Splitting dataset
train_X, test_X, train_y, test_y = train_test_split(X, y,
													test_size = 0.2,
													random_state = 42)

from sklearn.model_selection import GridSearchCV

param_grid = {
    'n_estimators': [100, 200],
    'learning_rate': [0.01, 0.05, 0.1],
    'max_depth': [3, 5],
    'max_features': [5, 'sqrt', 'log2'],
    'subsample': [0.8, 1.0],  # Fraction of samples to use for fitting each tree
    'min_samples_split': [2, 5],  # Minimum samples required to split an internal node
}

grid_search = GridSearchCV(GradientBoostingClassifier(random_state=42), param_grid, cv=5, scoring='accuracy', n_jobs=-1)
grid_search.fit(train_X, train_y)

# Best parameters from grid search
print(f"Best Parameters: {grid_search.best_params_}")

# Instantiate Gradient Boosting with best parameters
gbc = GradientBoostingClassifier(**grid_search.best_params_, random_state=42)

# Fit to training set
gbc.fit(train_X, train_y)

# Predict on test set
pred_y = gbc.predict(test_X)

# Accuracy
acc = accuracy_score(test_y, pred_y)
print(f"Gradient Boosting Classifier accuracy is : {acc}")

# Additional Evaluation Metrics
print("\nClassification Report:")
print(classification_report(test_y, pred_y))

print("\nConfusion Matrix:")
print(confusion_matrix(test_y, pred_y))

# Feature Importances (Optional)
importances = gbc.feature_importances_
print("\nFeature Importances:")
for feature, importance in zip(X.columns, importances):
    print(f"{feature}: {importance:.4f}")


# # Instantiate Gradient Boosting Regressor
# gbc = GradientBoostingClassifier(n_estimators=100,
# 								learning_rate=0.05,
# 								random_state=42,
# 								max_features=5 )
# # Fit to training set
# gbc.fit(train_X, train_y)

# # Predict on test set
# pred_y = gbc.predict(test_X)

# # accuracy
# acc = accuracy_score(test_y, pred_y)
# print(f"Gradient Boosting Classifier accuracy is : {acc}")

Best Parameters: {'learning_rate': 0.05, 'max_depth': 3, 'max_features': 'sqrt', 'min_samples_split': 5, 'n_estimators': 200, 'subsample': 1.0}
Gradient Boosting Classifier accuracy is : 0.974155069582505

Classification Report:
              precision    recall  f1-score   support

       False       0.98      1.00      0.99       466
        True       0.93      0.70      0.80        37

    accuracy                           0.97       503
   macro avg       0.95      0.85      0.89       503
weighted avg       0.97      0.97      0.97       503


Confusion Matrix:
[[464   2]
 [ 11  26]]

Feature Importances:
TransactionAmount: 0.2295
TransactionType: 0.0024
Location: 0.0084
Channel: 0.0012
CustomerAge: 0.0179
CustomerOccupation: 0.0061
TransactionDuration: 0.0386
LoginAttempts: 0.5878
AccountBalance: 0.0765
TimeSinceLastTransaction: 0.0317
